In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import datetime as dt
import calendar,warnings,itertools,matplotlib,keras,shutil
import tensorflow as tf
import statsmodels.api as sm
from datetime import datetime
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression,ElasticNet, Lasso
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier, GradientBoostingRegressor,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score,roc_curve, auc
from sklearn.datasets import load_iris,make_regression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.kernel_ridge import KernelRidge
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from IPython.core import display as ICD
warnings.filterwarnings('ignore') 

In [ ]:
#Importing Dataset using pandas
dataset=pd.read_csv("../input/dataco-smart-supply-chain-for-big-data-analysis/DataCoSupplyChainDataset.csv",header= 0,encoding= 'unicode_escape')
dataset.head(5)

In [ ]:
dataset.shape

In [ ]:
#Dropping Columns that are irrelevant and repetitive

## Concatenating first and last name so that one could be dropped
dataset['Customer Full Name'] = dataset['Customer Fname'].astype(str)+dataset['Customer Lname'].astype(str) 

data=dataset.drop(['Customer Email','Product Status','Customer Password','Customer Street','Customer Fname','Customer Lname',
           'Latitude','Longitude','Product Description','Product Image','Order Zipcode','shipping date (DateOrders)','Customer Zipcode', 'order date (DateOrders)','Delivery Status' ],axis=1)

In [ ]:
data.apply(lambda x: sum(x.isnull())) #Checking missing values

In [ ]:
train_data=data.copy()

# Heatmap for correlation matrix

In [ ]:
fig, ax = plt.subplots(figsize=(24,12)) # figsize
sns.heatmap(data.corr(),annot=True,linewidths=.5,fmt='.1g',cmap= 'coolwarm') # Heatmap for correlation matrix

In [ ]:
data.plot(x='Product Price', y='Sales per customer',linestyle='dotted',
     markerfacecolor='blue', markersize=12) 
plt.title('Product Price vs Sales per customer')#title
plt.xlabel('Product Price')  # X-axis title
plt.ylabel('Sales per customer') # Y=axis title
plt.show()

# Casestudy 1: Fraud Order Classification 

In [ ]:
# Creating a new column with binary classification of fraud status
train_data['fraud'] = np.where(train_data['Order Status'] == 'SUSPECTED_FRAUD', 1, 0)
train_data.drop(['Order Status'], axis=1, inplace=True)

In [ ]:
# Checking data tyoes so that relevant encosing couldcould be done 

train_data.dtypes

In [ ]:
# Using Label encoder all the column with datatype as object is been transformed

le = preprocessing.LabelEncoder()
train_data['Customer Country']  = le.fit_transform(train_data['Customer Country'])
train_data['Market']            = le.fit_transform(train_data['Market'])
train_data['Type']              = le.fit_transform(train_data['Type'])
train_data['Product Name']      = le.fit_transform(train_data['Product Name'])
train_data['Customer Segment']  = le.fit_transform(train_data['Customer Segment'])
train_data['Customer State']    = le.fit_transform(train_data['Customer State'])
train_data['Order Region']      = le.fit_transform(train_data['Order Region'])
train_data['Order City']        = le.fit_transform(train_data['Order City'])
train_data['Category Name']     = le.fit_transform(train_data['Category Name'])
train_data['Customer City']     = le.fit_transform(train_data['Customer City'])
train_data['Department Name']   = le.fit_transform(train_data['Department Name'])
train_data['Order State']       = le.fit_transform(train_data['Order State'])
train_data['Shipping Mode']     = le.fit_transform(train_data['Shipping Mode'])
train_data['Order Country']     = le.fit_transform(train_data['Order Country'])
train_data['Customer Full Name']= le.fit_transform(train_data['Customer Full Name'])

In [ ]:
def classifiermodel(model_f,model_l,xf_train, xf_test,yf_train,yf_test,xl_train, xl_test,yl_train,yl_test):
    model_f=model_f.fit(xf_train,yf_train) # Fitting train data for fraud detection
    model_l=model_l.fit(xl_train,yl_train) # Fitting train data for predection of late delivery
    yf_pred=model_f.predict(xf_test)
    yl_pred=model_l.predict(xl_test)  
    accuracy_f=accuracy_score(yf_pred, yf_test) #Accuracy for fraud detection
    accuracy_l=accuracy_score(yl_pred, yl_test) #Accuracy for predection of late delivery
    recall_f=recall_score(yf_pred, yf_test) #Recall score for  fraud detection
    recall_l=recall_score(yl_pred, yl_test)# Recall score for predection of late delivery
    conf_f=confusion_matrix(yf_test, yf_pred)# fraud detection
    conf_l=confusion_matrix(yl_test, yl_pred)#predection of late delivery
    f1_f=f1_score(yf_test, yf_pred)#fraud detection
    f1_l=f1_score(yl_test, yl_pred)#predection of late delivery
    print('Model paramters used are :',model_f)
    print('Accuracy of fraud status is        :', (accuracy_f)*100,'%')
    print('Recall score of fraud status is        :', (recall_f)*100,'%')
    print('Conf Matrix of fraud status is        :\n',  (conf_f))
    print('F1 score of fraud status is        :', (f1_f)*100,'%')
    print('Accuracy of late delivery status is:', (accuracy_l)*100,'%')
    print('Recall score of late delivery status is:', (recall_l)*100,'%')
    print('Conf Matrix of late delivery status is: \n',(conf_l))
    print('F1 score of late delivery status is:', (f1_l)*100,'%')

In [ ]:
keras.layers.BatchNormalization()
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(1024, activation='relu',kernel_initializer='random_normal', input_dim=38)) #Since we have 44 columns
classifier.add(Dropout(0.3))
#Third Hidden Layer
classifier.add(Dense(512, activation='relu',kernel_initializer='random_normal'))
classifier.add(Dropout(0.3))
#Fourth Hidden Layer
classifier.add(Dense(256, activation='relu',kernel_initializer='random_normal'))
classifier.add(Dropout(0.3))
#Fifth Hidden Layer
classifier.add(Dense(128, activation='relu',kernel_initializer='random_normal'))
classifier.add(Dropout(0.3))
#Output Layer
classifier.add(Dense(1, activation='sigmoid',kernel_initializer='random_normal'))


In [ ]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#All columns expect fraud
xf=train_data.loc[:, train_data.columns != 'fraud']
#Only fraud column
yf=train_data['fraud']
#Splitting the data into two parts in which 80% data will be used for training the model and 20% for testing
xf_train, xf_test,yf_train,yf_test = train_test_split(xf,yf,test_size = 0.2,random_state = 42)

In [ ]:
#Preparing vaidation set for finding the optimun number of epochs required.

xf_val = xf[:28000] # reserving the first 10,000 reviews for validation
xf_train_small = xf[28000:] # reserving the remaining 15,000 reviews for training
yf_val = yf[:28000]
yf_train_small = yf[28000:]
history = classifier.fit(xf_train_small, 
                    yf_train_small, 
                    epochs=20, 
                    batch_size=216,
                   validation_data=(xf_val, yf_val))

In [ ]:
history_dict = classifier.history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
classifier.fit(xf_train,yf_train, batch_size=216, epochs=3)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(classifier, to_file='Case1.png', show_shapes=True)

In [ ]:
train_evaluate=classifier.evaluate(xf_train, yf_train)
test_evaluate=classifier.evaluate(xf_test, yf_test)
print('accuracy for Train set is',train_evaluate)
print('accuracy for Test set is',test_evaluate)# evaluation of model.
yf_pred1=classifier.predict(xf_test,batch_size=512,verbose=1)
yf_pred=np.argmax(yf_pred1,axis=1)
print(f1_score(yf_test,yf_pred,average="weighted"))

# Casestudy 2: Late Delivery Classification 

In [ ]:
xl=train_data.loc[:, train_data.columns != 'Late_delivery_risk']
#Only fraud column
yl=train_data['Late_delivery_risk']
#Splitting the data into two parts in which 80% data will be used for training the model and 20% for testing
xl_train, xl_test,yl_train,yl_test = train_test_split(xl,yl,test_size = 0.1, random_state = 42)

In [ ]:
sc = StandardScaler()
xl_train=sc.fit_transform(xl_train)
xl_test=sc.transform(xl_test)

In [ ]:
xl_val = xl[:28000] # reserving the first 10,000 reviews for validation
xl_train_small = xl[28000:] # reserving the remaining 15,000 reviews for training
yl_val = yl[:28000]
yl_train_small = yl[28000:]
history = classifier.fit(xl_train_small, 
                    yl_train_small, 
                    epochs=20, 
                    batch_size=216,
                   validation_data=(xl_val, yl_val))

In [ ]:
history_dict = classifier.history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
classifier.fit(xl_train,yl_train, batch_size=216, epochs=5)

In [ ]:
train_evaluate=classifier.evaluate(xl_train, yl_train)
test_evaluate=classifier.evaluate(xl_test, yl_test)
print('accuracy for Train set is',train_evaluate)
print('accuracy for Test set is',test_evaluate)# evaluation of model.
yl_pred1=classifier.predict(xl_test,batch_size=216,verbose=1)
yl_pred=np.argmax(yl_pred1,axis=1)
print(f1_score(yl_test,yl_pred,average="weighted"))

keras.layers.BatchNormalization()
classifier_l= Sequential()
#First Hidden Layer
classifier_l.add(Dense(1024, activation='relu',kernel_initializer='random_normal', input_dim=38)) #Since we have 44 columns
classifier_l.add(Dropout(0.3))
#Third Hidden Layer
classifier_l.add(Dense(512, activation='relu',kernel_initializer='random_normal'))
classifier_l.add(Dropout(0.3))
#Fourth Hidden Layer
classifier_l.add(Dense(256, activation='relu',kernel_initializer='random_normal'))
classifier_l.add(Dropout(0.3))
#Fifth Hidden Layer
classifier_l.add(Dense(128, activation='relu',kernel_initializer='random_normal'))
classifier_l.add(Dropout(0.3))
#Sixth Hidden Layer
classifier_l.add(Dense(64, activation='relu',kernel_initializer='random_normal'))
classifier_l.add(Dropout(0.3))
#Seventh Hidden Layer
classifier_l.add(Dense(32, activation='relu',kernel_initializer='random_normal'))
classifier_l.add(Dropout(0.3))
#Eight Hidden Layer
classifier_l.add(Dense(16, activation='relu',kernel_initializer='random_normal'))
classifier_l.add(Dropout(0.3))
#Ninth Hidden Layer
classifier_l.add(Dense(8, activation='relu',kernel_initializer='random_normal'))
classifier_l.add(Dropout(0.3))
#Tenth Hidden Layer
classifier_l.add(Dense(4, activation='relu',kernel_initializer='random_normal'))
classifier_l.add(Dropout(0.3))
#Eleventh Hidden Layer
classifier_l.add(Dense(2, activation='relu',kernel_initializer='random_normal'))
classifier_l.add(Dropout(0.3))
#Output Layer
classifier_l.add(Dense(1, activation='sigmoid',kernel_initializer='random_normal'))

# Casestudy 3: Regression on Sales Data

In [ ]:
xs=train_data.loc[:, train_data.columns != 'Sales']
ys=train_data['Sales']
xs_train, xs_test,ys_train,ys_test = train_test_split(xs,ys,test_size = 0.3, random_state = 42)

In [ ]:
scaler=MinMaxScaler()
xs_train=scaler.fit_transform(xs_train)
xs_test=scaler.transform(xs_test)

In [ ]:
def regressionmodel(model_s,model_q,xs_train, xs_test,ys_train,ys_test,xq_train, xq_test,yq_train,yq_test):
    model_s=model_s.fit(xs_train,ys_train)#Fitting train data for sales
    model_q=model_q.fit(xq_train,yq_train)#Fitting train data for order quantity
    ys_pred=model_s.predict(xs_test)#predicting sales with test data
    yq_pred=model_q.predict(xq_test)#predicting order quantity with test data
    print('Model parameter used are:',model_s) #Printing the model to see which parameters are used
    #Printing mean absolute error for predicting sales
    print("MAE of sales is         :", metrics.mean_absolute_error(ys_test,ys_pred))
    #Printing Root mean squared error for predicting sales
    print("RMSE of sales is        :",np.sqrt(metrics.mean_squared_error(ys_test,ys_pred)))
    #Printing mean absolute error for predicting order quantity
    print("MAE of order quantity   :", metrics.mean_absolute_error(yq_test,yq_pred))
    #Printing Root mean squared error for predicting order quantity
    print("RMSE of order quantity  :",np.sqrt(metrics.mean_squared_error(yq_test,yq_pred)))

In [ ]:
regressor = Sequential()

#First Hidden Layer
regressor.add(Dense(512, activation='relu',kernel_initializer='normal',input_dim=38))
regressor.add(Dropout(0.3))
#Second  Hidden Layer
regressor.add(Dense(256, activation='relu',kernel_initializer='normal'))
regressor.add(Dropout(0.3))
#Third  Hidden Layer
regressor.add(Dense(256, activation='relu',kernel_initializer='normal'))
regressor.add(Dropout(0.3))
#Fourth  Hidden Layer
regressor.add(Dense(256, activation='relu',kernel_initializer='normal'))
regressor.add(Dropout(0.3))
#Fifth  Hidden Layer
regressor.add(Dense(256, activation='relu',kernel_initializer='normal'))
#Output Layer
regressor.add(Dense(1, activation='linear'))# Linear activation is used.

In [ ]:
X = array(X).reshape(20, 1, 1)

In [ ]:
#Build the LSTM network model
model = Sequential()
model.add(LSTM(units=512, dropout=0.5, recurrent_dropout=0.5,return_sequences=True, activation='relu',kernel_initializer='random_normal', input_shape=(38, 1)))
model.add(Dense(units=1,activation='linear'))


In [ ]:
xs_val = xs[:28000] # reserving the first 10,000 reviews for validation
xs_train_small = xs[28000:] # reserving the remaining 15,000 reviews for training
ys_val = ys[:28000]
ys_train_small = ys[28000:]
history = classifier.fit(xs_train_small, 
                    ys_train_small, 
                    epochs=30, 
                    batch_size=216,
                   validation_data=(xs_val, ys_val))

In [ ]:
history_dict = classifier.history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
regressor.compile(optimizer='rmsprop',loss='mean_absolute_error',metrics=['mean_absolute_error'])
regressor.fit(xs_train,ys_train, batch_size=216, epochs=30)

In [ ]:
print(xs_train.shape)
print(xs_test.shape)


In [ ]:
model.compile(optimizer='rmsprop',loss='mean_absolute_error',metrics=['mean_absolute_error'])
model.fit(xs_train,ys_train, batch_size=216, epochs=5)

In [ ]:
pred_train_s= regressor.predict(xs_train)
pred_s_test= regressor.predict(xs_test)
print('MAE Value train data:',regressor.evaluate(xs_train,ys_train))
print('RMSE of train data:',np.sqrt(mean_squared_error(ys_train,pred_train_s)))
print('MAE Value test data:',regressor.evaluate(xs_test,ys_test))
print('RMSE of test data:',np.sqrt(mean_squared_error(ys_test,pred_s_test)))

# LSTM

In [ ]:
from keras.layers import LSTM

In [ ]:
xt=train_data.loc[:, train_data.columns != 'Sales']
yt=train_data['Sales']
xt_train, xt_test,yt_train,yt_test = train_test_split(xt,yt,test_size = 0.3, random_state = 42)

In [ ]:
scaler=MinMaxScaler()
xt_train=scaler.fit_transform(xt_train)
xt_test=scaler.transform(xt_test)

In [ ]:
ytt = train_data['Sales'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
ytt_scaled = min_max_scaler.fit_transform(ytt)


In [ ]:
xt_train, xt_test,yt_train,yt_test = train_test_split(xt,ytt_scaled,test_size = 0.3, random_state = 42)

In [ ]:
yt_test

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
xt_train1 =xt_train.reshape(xt_train.shape[0],xt_train.shape[1] , 1)
xt_test1 = xt_test.reshape(xt_test.shape[0],xt_test.shape[1] , 1)

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
yt_train1 =yt_train.reshape(yt_train.shape[0],yt_train.shape[1] , 1)
yt_test1 = yt_test.reshape(yt_test.shape[0],yt_test.shape[1] , 1)

In [ ]:
#Build the LSTM network model
model = Sequential()
model.add(LSTM(units=512, dropout=0.5, recurrent_dropout=0.5,return_sequences=True, activation='relu',kernel_initializer='random_normal', input_shape=(38, 1)))
model.add(Dense(units=1,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_error'])

In [ ]:
model.fit(xt_train,yt_train, batch_size=216, epochs=5)

In [ ]:
xt_train.shape[0]

In [ ]:
train_predict=model.predict(xt_train1)
test_predict=model.predict(xt_test1)

In [ ]:
modeva= model.evaluate(xt_train1,yt_train)

In [ ]:
print('MAE Value train data:',modeva)

In [ ]:
modevatest=model.evaluate(xt_test1,yt_test)

In [ ]:
print('MAE Value test data:',modevatest)